# 00 · From SDE to Fokker–Planck, NESS, and Helmholtz split

This notebook connects continuous **stochastic dynamics** to the
Fokker–Planck viewpoint and the **Helmholtz (Hodge) decomposition** used by Friston’s
Bayesian mechanics narrative.

We:
1. Simulate a 2D Langevin SDE: $\mathrm{d}x_t = f(x_t)\,\mathrm{d}t + \sqrt{2D}\,\mathrm{d}w_t$.
2. Empirically estimate the **steady-state density** $p^*(x)$ (ergodic histogram) and potential $\Phi(x)=-\ln p^*(x)$.
3. Decompose the drift **on a periodic grid**: $f = \underbrace{\nabla\phi}_{\text{curl-free}} + \underbrace{\mathbf R\nabla\psi}_{\text{divergence-free}}$.

> **Fokker–Planck link (recap):**
$$\partial_t p(x,t) = -\nabla\cdot(f p) + \nabla\cdot D\,\nabla p.$$
At non-equilibrium steady state (NESS), $\partial_t p^*=0$; density can be stationary while probability **currents** circulate (solenoidal component).

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from persystems.sde import euler_maruyama, drift_doublewell_with_swirl
from persystems.fp import histogram2d_density, separable_gaussian_blur, potential_from_density
from persystems.helmholtz import helmholtz_decompose_periodic
np.random.seed(7)
plt.rcParams['figure.dpi'] = 120

## 1) Simulate a 2D Langevin SDE (double-well + swirl)
We use Euler–Maruyama with a potential-driven part (double well) plus a rotational field (solenoidal swirl).

In [ ]:
T  = 200_000
dt = 0.002
D  = 0.05
f  = drift_doublewell_with_swirl(a=1.0, b=1.0, k=1.0)
X  = euler_maruyama(np.array([0.0, 0.5]), f, dt=dt, T=T, D=D)
burn = int(0.2*T)
Xs = X[burn:]

plt.figure(figsize=(5,4))
plt.plot(Xs[::200,0], Xs[::200,1], '.', ms=1)
plt.title('Subsampled trajectory after burn-in')
plt.xlabel('x'); plt.ylabel('y'); plt.tight_layout(); plt.show()
Xs.shape

## 2) Empirical steady-state density and potential
We estimate $p^*(x)$ via a normalized 2D histogram (ergodic approximation), smooth it, and compute $\Phi=-\ln p^*(x)$ for visualization.
This is a practical surrogate for solving the stationary Fokker–Planck PDE on a grid.

In [ ]:
pad = 0.5
xmin, xmax = Xs[:,0].min()-pad, Xs[:,0].max()+pad
ymin, ymax = Xs[:,1].min()-pad, Xs[:,1].max()+pad
Nx, Ny = 120, 120
xbins = np.linspace(xmin, xmax, Nx+1)
ybins = np.linspace(ymin, ymax, Ny+1)
p, xc, yc = histogram2d_density(Xs, xbins, ybins)
p_s = separable_gaussian_blur(p, sigma_x=1.0, sigma_y=1.0)
Phi = potential_from_density(p_s)

XX, YY = np.meshgrid(xc, yc)
plt.figure(figsize=(6,5))
plt.imshow(p_s, origin='lower', extent=[xmin,xmax,ymin,ymax], aspect='auto')
plt.colorbar(label='p*(x)')
cs = plt.contour(XX, YY, Phi, levels=20, colors='k', linewidths=0.5, alpha=0.6)
plt.clabel(cs, inline=True, fmt='%.1f', fontsize=7)
plt.title('Empirical steady-state density and potential Φ = −log p*')
plt.xlabel('x'); plt.ylabel('y'); plt.tight_layout(); plt.show()

## 3) Drift field and Helmholtz (Hodge) decomposition on a periodic grid
On the grid we evaluate the drift $f$ and decompose it into curl-free (**gradient**) and divergence-free (**solenoidal**) components via a Fourier-space projector (periodic boundary assumption). The solenoidal part captures **circulating probability currents** possible at NESS.

In [ ]:
# Evaluate drift on the grid
Vx = np.zeros_like(XX)
Vy = np.zeros_like(YY)
for i in range(Ny):
    for j in range(Nx):
        v = f(np.array([XX[i,j], YY[i,j]]))
        Vx[i,j], Vy[i,j] = v[0], v[1]

# Helmholtz split (periodic)
vgx, vgy, vsx, vsy = helmholtz_decompose_periodic(Vx, Vy, Lx=(xmax-xmin), Ly=(ymax-ymin))
rec_err = float(np.sqrt(np.mean((Vx-(vgx+vsx))**2 + (Vy-(vgy+vsy))**2)))
rec_err

In [ ]:
skip = max(1, Nx//30)
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.quiver(XX[::skip,::skip], YY[::skip,::skip], vgx[::skip,::skip], vgy[::skip,::skip], scale=60, width=0.003)
plt.title('Helmholtz: gradient (curl-free) component')
plt.xlabel('x'); plt.ylabel('y')
plt.subplot(1,2,2)
plt.quiver(XX[::skip,::skip], YY[::skip,::skip], vsx[::skip,::skip], vsy[::skip,::skip], scale=60, width=0.003)
plt.title('Helmholtz: solenoidal (divergence-free) component')
plt.xlabel('x'); plt.ylabel('y')
plt.suptitle(f'Reconstruction RMSE ≈ {rec_err:.3e}')
plt.tight_layout(); plt.show()

### Interpretation
- **Gradient component** aligns with $-\nabla \Phi$ and reflects dissipative flow toward typical states ("entropy-resisting").
- **Solenoidal component** produces rotational probability currents that can persist at NESS even though $p^*$ is stationary.
- This clarifies why a system can be out of equilibrium (non-zero currents) yet have a time-invariant distribution.

**Next:** Notebook 01 constructs an approximate **Markov blanket** partition from samples and checks conditional independencies via precision-matrix blocks.